### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- get Global Human Settlement Layer population estimates data using the Google Earth Engine functionalities
- clip it to an AOI

### Prerequisites
- earth engine api (ee) https://developers.google.com/earth-engine/apidocs
- gee map (gee), https://geemap.org/ by Qiusheng Wu
- create an account and a Google Cloud project in Google Developer, see here: https://developers.google.com/earth-engine/guides/auth

# init

In [1]:
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import logging
# extract coordinates from the bounding box
def get_coords(gdf):
    coords = gdf.to_crs(epsg='4326').envelope
    epsg_coords = coords.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    coi = roi.centroid(maxError=1)
    return epsg_coords, roi, rec_roi, coi


In [2]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-alexandresantosgeographie') # Replace with your GEE project
print(ee.__version__)

0.1.406


In [3]:
# check if the authentication is working
print(ee.Image("JRC/GHSL/P2023A/GHS_POP/1975").getInfo())

{'type': 'Image', 'bands': [{'id': 'population_count', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [360820, 180000], 'crs': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [100, 0, -18041000, 0, -100, 9000000]}], 'version': 1690185742435464, 'id': 'JRC/GHSL/P2023A/GHS_POP/1975', 'properties': {'system:time_start': 157766400000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180, -90], [180, -90], 

# imports

In [4]:
#input a vector and a raster file
AOI_path = Path('../data/processed/')

AOI_file_JAK = 'JAK_LIM_BoundingBox_AOI_A.shp' #Jakarta
AOI_file_MUM = 'MUM_LIM_BoundingBox_AOI_A.shp' #Mumbai
AOI_file_MAN = 'MAN_LIM_BoundingBox_AOI_A.shp' #Manila

#read the vector files
AOI_gdf_JAK = gpd.read_file(AOI_path/AOI_file_JAK).to_crs(epsg=4326)
AOI_gdf_MUM = gpd.read_file(AOI_path/AOI_file_MUM).to_crs(epsg=4326)
AOI_gdf_MAN = gpd.read_file(AOI_path/AOI_file_MAN).to_crs(epsg=4326)

# get the coordinates
JAK_aoi_crs, JAK_roi, JAK_rec_roi, JAK_coi = get_coords(AOI_gdf_JAK)
MUM_aoi_crs, MUM_roi, MUM_rec_roi, MUM_coi = get_coords(AOI_gdf_MUM)
MAN_aoi_crs, MAN_roi, MAN_rec_roi, MAN_coi = get_coords(AOI_gdf_MAN)

print(JAK_aoi_crs,MUM_aoi_crs,MAN_aoi_crs)

4326 4326 4326


# get

In [8]:
# methods from https://custom-scripts.sentinel-hub.com/sentinel-2/false-color-urban-rgb/

def roi_false_color(aoi_gdf):
    aoi_coords = aoi_gdf.to_crs(epsg='4326').envelope
    #print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])
    transform = [aoi_coords.bounds.values[0][0], aoi_coords.bounds.values[0][1], aoi_coords.bounds.values[0][2], aoi_coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    
    s2=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filterDate('2020-01-01', '2020-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).mosaic()
    # Use median to reduce noise and clouds

    # Calculate NDVI
    false_color = s2
    #false_color = s2.add(['B12', 'B11']).rename('false_color')  
    #false_color = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')  

    return false_color

In [9]:
JAK_fc = roi_false_color(AOI_gdf_JAK)

JAK_fc

In [10]:
Map = geemap.Map()
vis_params = {'bands': ['B12', 'B11', 'B4'], 'min': 0.0, 'max': 3000, 'opacity': 1.0, 'gamma': 1.2}
Map.centerObject(JAK_roi, zoom=8)
Map.addLayer(JAK_fc, vis_params, "Landsat Vis")
Map

Map(center=[-6.356093939017252, 106.7482247859414], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
JAK_false_color = JAK_fc.select(['B12', 'B11', 'B4'])

In [12]:


# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(JAK_roi, 10)
Map.addLayer(JAK_fc.clip(JAK_roi), ndviParams, 'false color')
Map

EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [12]:
MUM_ndvi = roi_ndvi(AOI_gdf_MUM)

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(MUM_roi, 10)
Map.addLayer(MUM_ndvi.clip(MUM_roi), ndviParams, 'NDVI')
Map

Map(center=[19.034270314559958, 73.07882763939186], controls=(WidgetControl(options=['position', 'transparent_…

In [23]:
MAN_ndvi = roi_ndvi(AOI_gdf_MAN)

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(MAN_roi, 10)
Map.addLayer(MAN_ndvi.clip(MAN_roi), ndviParams, 'NDVI')
Map

Map(center=[14.49940177150628, 121.0648231854315], controls=(WidgetControl(options=['position', 'transparent_b…

In [32]:
# define a function that exports the image to Google Drive
def export_image(image, filename, roi): #crs, projection, removed these arguments as they caused a resampling error
    try: 
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=filename,
            folder='gee_exports',
            region=roi,
            #crs=crs,
            #crsTransform=projection['transform'],
            maxPixels=1e13,
            fileFormat='GeoTIFF',
            formatOptions={
                'cloudOptimized': False
            }
        )
        task.start()
        logging.info(f'Exporting {filename} to Google Drive')
        return task
    except Exception as e:
        logging.error(f'An error has occurred: {e}')
        return None

# check image stats before export
def check_image_stats(image, roi): 
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=250,
        maxPixels=1e9       
    )
    return stats.getInfo()


In [45]:
JAK_ndvi_clip = JAK_ndvi.clip(JAK_rec_roi)
MUM_ndvi_clip = MUM_ndvi.clip(MUM_rec_roi)
MAN_ndvi_clip = MAN_ndvi.clip(MAN_rec_roi)
print('JAK_NDVI stats:', check_image_stats(JAK_ndvi_clip, JAK_rec_roi))
print('MUM_NDVI stats:', check_image_stats(MUM_ndvi_clip, MUM_rec_roi))
print('MAN_NDVI stats:', check_image_stats(MAN_ndvi_clip, MAN_rec_roi))


JAK_NDVI stats: {'NDVI_max': 0.9062659195109526, 'NDVI_min': -0.6334365325077399}
MUM_NDVI stats: {'NDVI_max': 0.8241356382978723, 'NDVI_min': -0.4794520547945205}
MAN_NDVI stats: {'NDVI_max': 0.9682539682539683, 'NDVI_min': -0.9803921568627451}


In [51]:
# define a function that exports the image to Google Drive
def gee_to_tiff(input_layer, output_filename, roi):
    out_folder = 'gee_exports'
    #projection = input_layer.projection().getInfo()
    
    try:
        task = geemap.ee_export_image_to_drive(
            input_layer,
            description=output_filename,
            folder=out_folder,
            maxPixels=1e13, #ressamples to a low resolution
            region = roi,
            #crs = projection['crs'],
            fileFormat = 'GeoTIFF',
            scale = 10           
        )
        #task.start()
        logging.info(f'Exporting {output_filename} to Google Drive')
        return task 
    except Exception as e:
        logging.error(f'An error has occurred: {e}')
        return None

# export

In [58]:
input_layer = MUM_ndvi_clip
output_filename = 'MUM_ENV_NDVI_Sentinel2_2020'
roi = MUM_rec_roi
out_folder = 'gee_exports'

export = geemap.ee_export_image_to_drive(
            input_layer,
            description=output_filename,
            folder=out_folder,
            maxPixels=1e13, #ressamples to a low resolution
            region = roi,
            crs = 'EPSG:4326',
            fileFormat = 'GeoTIFF',
            scale = 10           
        )

AttributeError: 'NoneType' object has no attribute 'start'

In [52]:
JAK_task = gee_to_tiff(JAK_ndvi_clip, 'JAK_ENV_NDVI_Sentinel2_2000', JAK_rec_roi) 
MUM_task = gee_to_tiff(MUM_ndvi_clip, 'MUM_ENV_NDVI_Sentinel2_2000', MUM_rec_roi) 
MAN_task = gee_to_tiff(MAN_ndvi_clip, 'MAN_ENV_NDVI_Sentinel2_2000', MAN_rec_roi) 

# check task status
tasks = [JAK_task, MUM_task, MAN_task]
for task in tasks: 
    if task: 
        print(f'Task {task.status()["id"]} is {task.status()["state"]}')
    else:
        print('Task failed')


Task failed
Task failed
Task failed
